# Question 1

In [16]:
import pandas as pd
import numpy as np

######################## PARAMTERS STARTING HERE ################################
# Read the Excel file from the 'Demand' sheet
file_path = "OR113-2_midtermProject_data.xlsx"
df_demand = pd.read_excel(file_path, sheet_name="Demand")
N = df_demand.shape[0] - 1   # -1 because of the first row, +1 for indices' consistency
T = df_demand.shape[1] - 2  # -2 because of the first two columns, +1 for indices' consistency  
#print("N:", N, "T:", T)

# Display the dataframe to verify the data
I = np.zeros([N, T])
D = np.zeros([N, T])
I_0 = np.zeros([N])

for i in range(N):
    I_0[i] = df_demand.iloc[i+1, 1]
    for t in range(T):
        D[i, t] = df_demand.iloc[i+1, t+2]

#print("I_0:", I_0)
#print("D:", D)

# Read the Excel file from the 'In-transit' sheet
df_in_transit = pd.read_excel(file_path, sheet_name="In-transit")
for i in range(N):
    for t in range(df_in_transit.shape[1] - 1):
        I[i, t] = df_in_transit.iloc[i+1, t+1]
#print("I:", I)

# Read the Excel file from the 'Shipping cost' sheet
df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost")
J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")


C = {
    "H": np.zeros([N]),
    "P": np.zeros([N]),
    "V": np.zeros([N, J]),
    "Pr": np.zeros([N]), #Price
    "F": np.array([100, 80, 50]),
    "C": 2750,
}
V = np.zeros([N])
V_C = 30
for i in range(N):
    C["H"][i] = df_inventory_cost.iloc[i, 3]
    C["P"][i] = df_inventory_cost.iloc[i, 2]
    C["Pr"][i] = df_inventory_cost.iloc[i, 1] #Price
    V[i] = df_shipping_cost.iloc[i, 3]
    for j in range(J):
        if j == J - 1:
            C["V"][i, j] = 0
        else:
            C["V"][i, j] = df_shipping_cost.iloc[i, j+1]

#print("C:", C)
#print("V:", V)
T_lead = np.array([1, 2, 3]) # T_j

In [19]:


######################## PARAMTERS ENDING HERE ##################################

import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t
s = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="s") # Number of lost quantity s_it

# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost + Lost cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)
# Lost-sales cost
lost_cost = gp.quicksum(
    (C["Pr"][i] - C["P"][i]) * s[i, t]
    for i in S_I for t in S_T
)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost + lost_cost, GRB.MINIMIZE)

# Constraints
# Inventory balance (2)
J_in_inventory = np.array([1, 2, 3, 3, 3, 3])

        
for i in S_I:
    for t in S_T:

        # 2-1 到 t 期末會抵達的在途訂單
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            if t - T_lead[j] < 0: continue
            in_inventory += x[i, j,  t - T_lead[j]]

        # 2-2 庫存平衡 (含缺貨量 s_it)
        if t == 0:
            model.addConstr(
                v[i, t] - s[i, t] == I_0[i] + I[i, t] + in_inventory - D[i, t],
                name=f"InvBal_{i}_{t}"
            )
        else:
            model.addConstr(
                v[i, t] - s[i, t] == v[i, t-1] + I[i, t] + in_inventory - D[i, t],
                name=f"InvBal_{i}_{t}"
            )
        
        # 2-3 缺貨上下界  0 ≤ s_it ≤ D_it
        model.addConstr(s[i, t] >= 0, name=f"NonNeg_s_{i}_{t}")
        model.addConstr(s[i, t] <= D[i, t], name=f"Upper_s_{i}_{t}")
        if (t != 0) :
            model.addConstr(v[i, t-1] >= D[i, t] - s[i, t], name=f"Upper_s_{i}_{t}")


M_jt = np.zeros((J, T))               # 2-D array  [j,t]

for j in S_J:
    for t in S_T:
        # 到貨週期 = t + T_lead[j] - 1  之後所有尚未發生的需求總和
        arrival = t + T_lead[j] - 1
        if arrival < T:               # arrival 超過規劃期末就保持 0
            M_jt[j, t] = D[:, arrival:].sum()
# ----------------------------
# 啟動 y_jt 之 Big-M 連結
# ----------------------------
for j in S_J:
    for t in S_T:
        model.addConstr(
            gp.quicksum(x[i, j, t] for i in S_I) <= M_jt[j, t] * y[j, t],
            name=f"ShipMethod_{j}_{t}"
        )


# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()

# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
            if v[i, t].x > 0:
                print(f"v[{i+1},{t+1}] = {v[i, t].x}")
                    
    # --------------------- 缺貨量 s_it ---------------------
    print("\nLost-sales quantity (s_it):")
    for t in S_T:
        for i in S_I:
            if s[i, t].x > 0:
                print(f"s[{i+1},{t+1}] = {s[i, t].x}")
                
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")

Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  0

Optimize a model with 518 rows, 324 columns and 1035 nonzeros
Model fingerprint: 0xbd954bbb
Variable types: 300 continuous, 24 integer (18 binary)
Coefficient statistics:
  Matrix range     [5e-03, 7e+03]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+02]
Found heuristic solution: objective 2.920518e+07
Presolve removed 482 rows and 191 columns
Presolve time: 0.00s
Presolved: 36 rows, 133 columns, 259 nonzeros
Found heuristic solution: objective 2.615349e+07
Variable types: 122 continuous, 11 integer (9 binary)

Root relaxation: objective 1.699436e+07, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node 

# Question 2

### PARAMTERS STARTING HERE

In [170]:
import pandas as pd
import numpy as np

In [171]:
# Read the Excel file from the 'Demand' sheet
file_path = "OR113-2_midtermProject_data.xlsx"
df_demand = pd.read_excel(file_path, sheet_name="Demand")
N = df_demand.shape[0] - 1   # -1 because of the first row, +1 for indices' consistency
T = df_demand.shape[1] - 2  # -2 because of the first two columns, +1 for indices' consistency  
#print("N:", N, "T:", T)

In [172]:
# Display the dataframe to verify the data
I = np.zeros([N, T])
D = np.zeros([N, T])
I_0 = np.zeros([N])

for i in range(N):
    I_0[i] = df_demand.iloc[i+1, 1]
    for t in range(T):
        D[i, t] = df_demand.iloc[i+1, t+2]

#print("I_0:", I_0)
#print("D:", D)

In [173]:
# Read the Excel file from the 'In-transit' sheet
df_in_transit = pd.read_excel(file_path, sheet_name="In-transit")
for i in range(N):
    for t in range(df_in_transit.shape[1] - 1):
        I[i, t] = df_in_transit.iloc[i+1, t+1]
#print("I:", I)

In [174]:
# Read the Excel file from the 'Shipping cost' sheet
df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost")
J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")

In [175]:
C = {
    "H": np.zeros([N]),
    "P": np.zeros([N]),
    "V": np.zeros([N, J]),
    "Pr": np.zeros([N]), #Price
    "F": np.array([100, 80, 50]),
    "C": 2750,
    ######## modified
    "B": np.zeros([N]),
    "prob": np.zeros([N])
    ######## modified
}
V = np.zeros([N])
V_C = 30
for i in range(N):
    C["H"][i] = df_inventory_cost.iloc[i, 3]
    C["P"][i] = df_inventory_cost.iloc[i, 2]
    C["Pr"][i] = df_inventory_cost.iloc[i, 1] #Price
    V[i] = df_shipping_cost.iloc[i, 3]
    for j in range(J):
        if j == J - 1:
            C["V"][i, j] = 0
        else:
            C["V"][i, j] = df_shipping_cost.iloc[i, j+1]
    ######## modified
    C["B"][i] = df_inventory_cost.iloc[i, 4]
    C["prob"][i] = df_inventory_cost.iloc[i, 5]
    ######## modified

#print("C:", C)
#print("V:", V)
T_lead = np.array([1, 2, 3]) # T_j

### Optimization

In [176]:
import gurobipy as gp
from gurobipy import GRB

In [177]:
# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
d = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="d")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t
######## modified
s = model.addVars(range(N+1), S_T, vtype=GRB.CONTINUOUS, name="s") # Number of lost quantity s_it
######## modified

Set parameter MIPGap to value 0


In [178]:
# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost + Lost cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)
######## modified
# Lost-sales cost
lost_cost = gp.quicksum(
    (C["Pr"][i] - C["P"][i]) * (1 - C["prob"][i]) * s[i, t]
    for i in S_I for t in S_T
)
backorder_discount = gp.quicksum(C["B"][i] * C["prob"][i] * s[i, t] for i in S_I for t in S_T)
######## modified

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost + lost_cost + backorder_discount, GRB.MINIMIZE)

In [179]:
# Constraints
# Inventory balance (2)
J_in_inventory = np.array([1, 2, 3, 3, 3, 3])

        
for i in S_I:
    for t in S_T:

        # 2-1 到 t 期末會抵達的在途訂單
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j,  t - T_lead[j]+1]

        # 2-2 庫存平衡 (含缺貨量 s_it)
        if t == 0:
            ######## modified
            model.addConstr(v[i, t] - s[i, t] == I_0[i] + I[i, t] + in_inventory - D[(i, t)], name=f"bal_{i}_{t}")
            ######## modified
        else:
            ######## modified
            model.addConstr(d[i, t] == D[(i, t)] + C["prob"][i] * s[i, t - 1], name=f"bal_{i}_{t}")
            model.addConstr(v[i, t] - s[i, t] == v[i, t-1] + I[i, t] + in_inventory - d[i, t], name=f"bal_{i}_{t}")
            model.addConstr(v[i, t-1] >= d[i, t] - s[i, t], name=f"Upper_s_{i}_{t}")
            ######## modified
        # 2-3 缺貨上下界  0 ≤ s_it ≤ D_it
        model.addConstr(s[i, t] >= 0, name=f"NonNeg_s_{i}_{t}")
        model.addConstr(s[i, t] <= d[i, t], name=f"Upper_s_{i}_{t}")
        # if (t != 0) :
        #     model.addConstr(v[i, t-1] >= D[i, t] - s[i, t], name=f"Upper_s_{i}_{t}")


M_jt = np.zeros((J, T))               # 2-D array  [j,t]

for j in S_J:
    for t in S_T:
        # 到貨週期 = t + T_lead[j] - 1  之後所有尚未發生的需求總和
        arrival = t + T_lead[j] - 1
        if arrival < T:               # arrival 超過規劃期末就保持 0
            M_jt[j, t] = D[:, arrival:].sum()
# ----------------------------
# 啟動 y_jt 之 Big-M 連結
# ----------------------------
for j in S_J:
    for t in S_T:
        model.addConstr(
            gp.quicksum(x[i, j, t] for i in S_I) <= M_jt[j, t] * y[j, t],
            name=f"ShipMethod_{j}_{t}"
        )


# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  0

Optimize a model with 568 rows, 390 columns and 1320 nonzeros
Model fingerprint: 0x8e8ab5d0
Variable types: 366 continuous, 24 integer (18 binary)
Coefficient statistics:
  Matrix range     [5e-03, 7e+03]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+02]
Found heuristic solution: objective 2.200800e+07
Presolve removed 490 rows and 222 columns
Presolve time: 0.00s
Presolved: 78 rows, 168 columns, 401 nonzeros
Found heuristic solution: objective 2.056894e+07
Variable types: 153 continuous, 15 integer (12 binary)

Root relaxation: objective 1.385294e+07, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      

In [180]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
            if v[i, t].x > 0:
                print(f"v[{i+1},{t+1}] = {v[i, t].x}")
                    
    # --------------------- 缺貨量 s_it ---------------------
    print("\nUnsatisfied-sales quantity (s_it):")
    for t in S_T:
        for i in S_I:
            if s[i, t].x > 0:
                print(f"s[{i+1},{t+1}] = {s[i, t].x}")
                
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")


Optimal objective value: 13854779.57142857

Order quantities (x_ijt):
x[8,3,1] = 61.0
x[10,1,1] = 24.0
x[10,2,1] = 162.0
x[10,3,1] = 200.0
x[3,3,2] = 82.0
x[8,3,2] = 168.0
x[10,3,2] = 129.57142857142858
x[3,3,3] = 200.0
x[6,3,3] = 144.0
x[8,3,3] = 32.0
x[9,3,3] = 94.0
x[10,2,3] = 24.428571428571427
x[1,2,4] = 38.0
x[6,2,4] = 10.0

Ending inventory (v_it):
v[1,1] = 662.0
v[2,1] = 458.0
v[3,1] = 346.0
v[4,1] = 361.0
v[5,1] = 365.0
v[6,1] = 451.0
v[7,1] = 376.0
v[8,1] = 181.0
v[9,1] = 674.0
v[10,1] = 178.0
v[1,2] = 607.0
v[2,2] = 357.0
v[3,2] = 187.0
v[4,2] = 258.0
v[5,2] = 303.0
v[6,2] = 379.0
v[7,2] = 257.0
v[8,2] = 26.0
v[9,2] = 523.0
v[10,2] = 162.0
v[1,3] = 435.0
v[2,3] = 289.0
v[3,3] = 166.0
v[4,3] = 180.0
v[5,3] = 220.0
v[6,3] = 272.0
v[7,3] = 238.0
v[8,3] = 77.0
v[9,3] = 335.0
v[10,3] = 200.0
v[1,4] = 241.0
v[2,4] = 104.0
v[3,4] = 199.0
v[4,4] = 49.0
v[5,4] = 130.0
v[6,4] = 145.0
v[7,4] = 122.0
v[8,4] = 168.0
v[9,4] = 159.0
v[10,4] = 154.0
v[1,5] = 185.0
v[2,5] = 91.0
v[3,5] = 20